### Import Packages

In [145]:
import pandas as pd
import requests
import sqlite3
import random
import json

In [50]:
user_agents = open('user_agent_list.txt', 'r').read().split('\n')

In [17]:
def get_request(request_url):
    response = requests.get(
        url=request_url,
        headers={'user-agent':  random.choice(user_agents)}
    )
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        return "Error: " + str(e)
    return response

In [ ]:
# Define wine type ids as defined by vivino.com

wine_type_ids = {
    'red': 1,
    'white': 2,
    'sparkling_wine': 3,
    'rose': 4,
    'liqueur_wine': 24,
    'dessert_wine': 7
}

In [ ]:
'https://www.vivino.com/api/wines/11890/reviews?per_page=10&page=1&year=2017&language=de'

In [140]:
response = get_request(request_url='https://www.vivino.com/api/explore/explore?wine_type_ids[]=1&per_page=20&page=1')
red_wines = response.json()
red_wines

{'explore_vintage': {'market': {'country': 'de',
   'currency': {'code': 'EUR', 'name': 'Euro', 'prefix': '€', 'suffix': None}},
  'records_matched': 47129,
  'matches': [{'vintage': {'id': 145279917,
     'seo_name': 'schrader-cellars-cabernet-sauvignon-rbs-beckstoffer-to-kalon-vineyard-2016',
     'name': 'Schrader Cabernet Sauvignon RBS Beckstoffer To Kalon Vineyard 2016',
     'statistics': {'status': 'Normal',
      'ratings_count': 99,
      'ratings_average': 4.9,
      'labels_count': 956,
      'wine_ratings_count': 1986,
      'wine_ratings_average': 4.7,
      'wine_status': 'Normal'},
     'image': {'location': '//images.vivino.com/thumbs/GpcSXs2ERS6niDxoAsvESA_pl_480x640.png',
      'variations': {'bottle_large': '//images.vivino.com/thumbs/GpcSXs2ERS6niDxoAsvESA_pb_x960.png',
       'bottle_medium': '//images.vivino.com/thumbs/GpcSXs2ERS6niDxoAsvESA_pb_x600.png',
       'bottle_medium_square': '//images.vivino.com/thumbs/GpcSXs2ERS6niDxoAsvESA_pb_600x600.png',
       'bot

In [144]:
red_wines['explore_vintage']['matches'][0]['vintage']['wine']

{'id': 1182309,
 'name': 'Cabernet Sauvignon RBS Beckstoffer To Kalon Vineyard',
 'seo_name': 'cabernet-sauvignon-rbs-beckstoffer-to-kalon-vineyard',
 'type_id': 1,
 'vintage_type': 0,
 'is_natural': False,
 'region': {'id': 42,
  'name': 'Oakville',
  'name_en': '',
  'seo_name': 'oak-ville',
  'country': {'code': 'us',
   'name': 'Vereinigte Staaten',
   'native_name': 'United States',
   'seo_name': 'united-states',
   'currency': {'code': 'USD',
    'name': 'US-Dollar',
    'prefix': '$',
    'suffix': None},
   'regions_count': 361,
   'users_count': 11662390,
   'wines_count': 255379,
   'wineries_count': 27865,
   'most_used_grapes': [{'id': 2,
     'name': 'Cabernet Sauvignon',
     'seo_name': 'cabernet-sauvignon',
     'has_detailed_info': True,
     'wines_count': 801751},
    {'id': 14,
     'name': 'Pinot Noir',
     'seo_name': 'pinot-noir',
     'has_detailed_info': True,
     'wines_count': 572334},
    {'id': 5,
     'name': 'Chardonnay',
     'seo_name': 'chardonnay',

In [290]:
con = sqlite3.connect('wine.db')
cur = con.cursor()
cur.execute('CREATE TABLE wine(id, name, type_id, is_natural, winemaker, region, country, grapes)')

In [291]:
wine_data = []
for i in range(20):
    grapes = []
    try:
        for grape_index in range(len(red_wines['explore_vintage']['matches'][i]['vintage']['wine']['style']['grapes'])):
            grapes.append(red_wines['explore_vintage']['matches'][i]['vintage']['wine']['style']['grapes'][grape_index]['name'])
        grapes = ', '.join(grapes)
    except:
        grapes = None
    wine_data.append(
        (
            red_wines['explore_vintage']['matches'][i]['vintage']['wine']['id'],
            red_wines['explore_vintage']['matches'][i]['vintage']['wine']['name'],
            red_wines['explore_vintage']['matches'][i]['vintage']['wine']['type_id'],
            red_wines['explore_vintage']['matches'][i]['vintage']['wine']['is_natural'],
            red_wines['explore_vintage']['matches'][i]['vintage']['wine']['winery']['name'],
            red_wines['explore_vintage']['matches'][i]['vintage']['wine']['region']['name'],
            red_wines['explore_vintage']['matches'][i]['vintage']['wine']['region']['country']['name'],
            grapes
        )
    )
cur.executemany('INSERT INTO wine VALUES(?, ?, ?, ?, ?, ?, ?, ?)', wine_data)
con.commit()

In [287]:
wine_data

[(1182309,
  'Cabernet Sauvignon RBS Beckstoffer To Kalon Vineyard',
  1,
  False,
  'Schrader',
  'Oakville',
  'Vereinigte Staaten',
  'Cabernet Sauvignon')]

In [239]:
test = 'leon' + ' Becker' + ' Michael'
test + ' ist'

'leon Becker Michael ist'

In [265]:
test += ' test'
test

'leon Becker Michaelhässlich hässlich hässlich hässlich hässlich hässlich hässlich hässlich hässlich test test test test test'

In [257]:
test

'leon Becker Michaelhässlich hässlich hässlich hässlich hässlich hässlich hässlich hässlich hässlich'

In [ ]:
# https://www.vivino.com/api/explore/explore?country_code=DE&currency_code=EUR&grape_filter=varietal&min_rating=3.8&order_by=&order=&price_range_max=30&price_range_min=7&wine_type_ids%5B%5D=1&page=1&language=de